# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298416867159                   -0.85    5.1
  2   -8.300179407153       -2.75       -1.26    1.0
  3   -8.300447212629       -3.57       -1.89    3.5
  4   -8.300461595015       -4.84       -2.77    1.5
  5   -8.300464083784       -5.60       -3.05    6.6
  6   -8.300464347657       -6.58       -3.23    1.0
  7   -8.300464493102       -6.84       -3.37    6.5
  8   -8.300464567093       -7.13       -3.49    1.2
  9   -8.300464625654       -7.23       -3.70    1.8
 10   -8.300464638242       -7.90       -3.88    1.0
 11   -8.300464643683       -8.26       -4.25    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.66862670997                   -0.70    6.4
  2   -16.67852000235       -2.00       -1.14    2.4
  3   -16.67921412518       -3.16       -1.86    3.9
  4   -16.67927473633       -4.22       -2.72    2.5
  5   -16.67928574389       -4.96       -3.11    5.9
  6   -16.67928620651       -6.33       -3.50    2.2
  7   -16.67928622004       -7.87       -3.96    1.2
  8   -16.67928622409       -8.39       -4.56    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32510102349                   -0.56    6.1
  2   -33.33261302676       -2.12       -1.00    1.6
  3   -33.33413072461       -2.82       -1.74    4.1
  4   -33.33429327252       -3.79       -2.64    4.8
  5   -33.33596479239       -2.78       -2.42    7.4
  6   -33.33694332406       -3.01       -2.52    6.8
  7   -33.33694322898   +   -7.02       -2.53    1.0
  8   -33.33686865097   +   -4.13       -2.34    2.0
  9   -33.33686963988       -6.00       -2.35    1.0
 10   -33.33687665466       -5.15       -2.37    1.0
 11   -33.33688995535       -4.88       -2.42    1.0
 12   -33.33689687775       -5.16       -2.45    1.0
 13   -33.33690990677       -4.89       -2.52    1.0
 14   -33.33693192761       -4.66       -2.75    1.6
 15   -33.33693331880       -5.86       -2.77    1.1
 16   -33.33694105324       -5.11       -3.05    3.8
 17   -33.33694389799       -5.55       -4.08 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298340971964                   -0.85    5.0
  2   -8.300241898110       -2.72       -1.59    1.0
  3   -8.300446236905       -3.69       -2.53    4.5
  4   -8.300364181518   +   -4.09       -2.26   14.8
  5   -8.300464373404       -4.00       -3.58    3.0
  6   -8.300464579193       -6.69       -3.90    3.2
  7   -8.300464640611       -7.21       -4.51    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32575791156                   -0.56    6.5
  2   -33.31820594871   +   -2.12       -1.27    1.0
  3   -17.46138794442   +    1.20       -0.47    8.1
  4   -33.32333180011        1.20       -1.77    8.0
  5   -33.04887096244   +   -0.56       -1.21    4.2
  6   -31.58514864306   +    0.17       -0.94    4.5
  7   -33.33202493401        0.24       -2.12    4.5
  8   -33.33006047073   +   -2.71       -1.88    2.9
  9   -33.33674762830       -2.17       -2.65    2.5
 10   -33.33671855591   +   -4.54       -2.83    2.8
 11   -33.33690512273       -3.73       -3.20    2.4
 12   -33.33694212782       -4.43       -3.66    3.6
 13   -33.33694271362       -6.23       -3.86    3.0
 14   -33.33694377098       -5.98       -4.22    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.